## Creating schema for Spotify data

In [1]:
import pandas as pd
import sqlite3

In [2]:
# read sptify data
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
spotify_songs.head(5)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [3]:
spotify_songs.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

### 1NF Normalization

The main purpose of this project is to create a normalized data schema for the data we just read in. There are three types of normalizations for general purposes. By 1NF normalization, Every table should not have any duplication or dependencies that are not key or domain constraints. We can see that there are no composite entries in the dataframe, so we are already 1NF-normalization-good. 

### 2NF Normalization

For 2NF normalization, we need to:  
- Break partial dependencies
    - Identify candidate Primary Key for each row
    - If there is a composite PK, see if other columns have partial dependencies

In [4]:
# composite PK: track_id, track_album_id, playlist_id
track_df = spotify_songs[['track_id', 'track_name', 'track_artist', 'track_popularity', 
                       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
track_album_df = spotify_songs[['track_album_id', 'track_album_name', 'track_album_release_date']]
playlist_df = spotify_songs[['playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre']]

### 3NF Normalization

For 3NF normalization:  
- Remove transitive dependencies, which means everything should depend on PK directly. 

In [5]:
track = track_df[['track_id', 'track_name', 'track_popularity', 'danceability', 'energy', 
                  'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
                  'liveness', 'valence', 'tempo', 'duration_ms']].drop_duplicates(subset=['track_id'])
album = track_album_df.drop_duplicates(subset=['track_album_id'])
track_album = spotify_songs[['track_id', 'track_album_id']].drop_duplicates()
playlist = playlist_df.drop_duplicates(subset=['playlist_id'])
track_playlist = spotify_songs[['track_id', 'playlist_id']].drop_duplicates()
artist = track_df[['track_id', 'track_artist']].drop_duplicates()

In [6]:
track.shape

(28356, 15)

In [7]:
album.head(5)

,track_album_id,track_album_name,track_album_release_date
0,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14
1,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13
2,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05
3,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19
4,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05


In [8]:
artist.head(5)

,track_id,track_artist
0,6f807x0ima9a1j3VPbc7VN,Ed Sheeran
1,0r7CVbZTWZgbTCYdfa2P31,Maroon 5
2,1z1Hg7Vb0AhHDiEmnDE79l,Zara Larsson
3,75FpbthrwQmzHlBJLuGdC7,The Chainsmokers
4,1e8PAfcKUYoKkxPhrHqw4x,Lewis Capaldi


In [9]:
playlist.shape

(471, 4)

The normalized tables are described as follows:   

- `track`: track_id, name, popularity and track characteristics;  
- `album`: track_album_id, name and release_date; 
- `artist`: track_id, track_artist;  
- `playlist`: playlist_id, name, genre and subgenre;  
- `track_album`: track_id and track_album_id;  
- `track_playlist`: track_id and playlist_id;  


### Creating database

In [10]:
# Create database
con = sqlite3.connect('spotify.db')

# Create tables in the database
track.to_sql(name='track', con=con, index = False)
album.to_sql(name='album', con=con, index = False)
artist.to_sql(name='artist', con=con, index = False)
playlist.to_sql(name='playlist', con=con, index = False)
track_album.to_sql(name='track_album', con=con, index = False)
track_playlist.to_sql(name='track_playlist', con=con, index = False)

In [11]:
%load_ext sql

In [12]:
%sql sqlite:///spotify.db

In [13]:
%%sql
select * from sqlite_master WHERE type='table'

 * sqlite:///spotify.db
Done.


type,name,tbl_name,rootpage,sql
table,track,track,2,"CREATE TABLE ""track"" (""track_id"" TEXT, ""track_name"" TEXT, ""track_popularity"" INTEGER, ""danceability"" REAL, ""energy"" REAL, ""key"" INTEGER, ""loudness"" REAL, ""mode"" INTEGER, ""speechiness"" REAL, ""acousticness"" REAL, ""instrumentalness"" REAL, ""liveness"" REAL, ""valence"" REAL, ""tempo"" REAL, ""duration_ms"" INTEGER)"
table,album,album,956,"CREATE TABLE ""album"" (""track_album_id"" TEXT, ""track_album_name"" TEXT, ""track_album_release_date"" TEXT)"
table,artist,artist,1282,"CREATE TABLE ""artist"" (""track_id"" TEXT, ""track_artist"" TEXT)"
table,playlist,playlist,1566,"CREATE TABLE ""playlist"" (""playlist_name"" TEXT, ""playlist_id"" TEXT, ""playlist_genre"" TEXT, ""playlist_subgenre"" TEXT)"
table,track_album,track_album,1576,"CREATE TABLE ""track_album"" (""track_id"" TEXT, ""track_album_id"" TEXT)"
table,track_playlist,track_playlist,1943,"CREATE TABLE ""track_playlist"" (""track_id"" TEXT, ""playlist_id"" TEXT)"


### Playlists that contain instrumentals

Below is the sql command to generate the name of all playlists that contain instrumentals. I will classify those whose instrumentalness greater than 0.1 to be containing instrumentals. 

In [22]:
## find the names of all playlists that contain instrumentals. 
%%sql
SELECT COUNT(DISTINCT(track.track_id)) AS num_tracks, playlist.playlist_id, playlist.playlist_name
FROM track
INNER JOIN track_playlist ON track_playlist.track_id = track.track_id
INNER JOIN playlist ON track_playlist.playlist_id = playlist.playlist_id
WHERE track.instrumentalness > 0.1
GROUP BY playlist.playlist_id
ORDER BY num_tracks DESC

 * sqlite:///spotify.db
Done.


num_tracks,playlist_id,playlist_name
96,37i9dQZF1DXc8kgYqQLMfH,Lush Lofi
93,72r6odw0Q3OWTCYMGA7Yiy,House Electro 2019
92,37i9dQZF1DX0SM0LYsmbMT,Jazz Vibes
91,29jj7pQlDqnWclbHQk21Rq,Underground Party | Hypnotic | Minimal | Acid | Big Room | Tech | Liquid
88,37i9dQZF1DWWQRwui0ExPn,Lo-Fi Beats
85,37i9dQZF1DX36Xw4IJIVKA,Lofi Hip-Hop
79,6nZaTh6K1SwhdELFTmA99C,Electro Posé - Discoveries
78,2JPzPB9jnvJLAYtmCbvZy8,Selected House
76,0AFYmoSuoMQiGGjzvBwr6u,💊ELECTRO-HOUSE-TECH💊
70,6KnQDwp0syvhfHOR4lWP7x,Fitness Workout Electro | House | Dance | Progressive House
